## 1. Install Required Packages

In [ ]:
# First install portaudio via homebrew: brew install portaudio
%pip install azure-ai-voicelive azure-identity azure-search-documents python-dotenv pyaudio

## 2. Import Libraries and Setup

In [ ]:
from __future__ import annotations
import os
import asyncio
import json
from datetime import datetime
import logging
from typing import Union, Optional, TYPE_CHECKING, cast, List, Dict, Any, Callable, Awaitable

from azure.core.credentials import AzureKeyCredential
from azure.core.credentials_async import AsyncTokenCredential
from azure.identity.aio import AzureCliCredential

from azure.ai.voicelive.aio import connect
from azure.ai.voicelive.models import (
    AudioEchoCancellation,
    AudioInputTranscriptionOptions,
    AudioNoiseReduction,
    AzureSemanticVad,
    AzureStandardVoice,
    InputAudioFormat,
    Modality,
    OutputAudioFormat,
    RequestSession,
    ServerEventType,
    ServerVad,
    FunctionCallOutputItem,
)
from dotenv import load_dotenv
import pyaudio

# Import reusable modules from local files
from audio_processor import AudioProcessor

# Import RAG tools - both AI Search and Redis backends
from rag_tools import RAGToolManager, create_rag_tools_from_env, ToolResult, ToolResultDirection
from rag_tools_redis import RedisRAGToolManager, create_redis_rag_tools_from_env

if TYPE_CHECKING:
    from azure.ai.voicelive.aio import VoiceLiveConnection

# Load environment variables
load_dotenv('./.env', override=True)

# Setup logging
if not os.path.exists('logs'):
    os.makedirs('logs')

timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
logging.basicConfig(
    filename=f'logs/{timestamp}_rag_assistant.log',
    filemode="w",
    format='%(asctime)s:%(name)s:%(levelname)s:%(message)s',
    level=logging.INFO
)
logger = logging.getLogger(__name__)
print("✅ Libraries imported and logging configured")
print("   Available RAG backends: 'ai_search', 'redis'")

## 3. RAG Voice Assistant Class

This class extends the basic voice assistant with tool/function calling capabilities for RAG.

In [ ]:
class RAGVoiceAssistant:
    """
    Voice assistant with RAG (Retrieval-Augmented Generation) capabilities.
    
    Extends the basic voice assistant to support:
    - Tool/function calling for knowledge base search
    - Source citation and grounding
    - Multi-turn conversations with context
    - Multiple RAG backends: Azure AI Search or Redis
    """

    def __init__(
        self,
        endpoint: str,
        credential: Union[AzureKeyCredential, AsyncTokenCredential],
        model: str,
        voice: str,
        instructions: str,
        rag_tools: Union[RAGToolManager, RedisRAGToolManager],
        voice_temperature: Optional[float] = None,
        input_audio_transcription_enabled: bool = False,
        input_audio_transcription_model: Optional[str] = None,
        vad_config: Optional[Dict] = None,
    ):
        self.endpoint = endpoint
        self.credential = credential
        self.model = model
        self.voice = voice
        self.instructions = instructions
        self.rag_tools = rag_tools
        self.voice_temperature = voice_temperature
        self.input_audio_transcription_enabled = input_audio_transcription_enabled
        self.input_audio_transcription_model = input_audio_transcription_model
        self.vad_config = vad_config or {}
        
        # Auto-detect language from voice
        if voice and "-" in voice:
            parts = voice.split("-")
            if len(parts) >= 2:
                self.transcription_language = f"{parts[0]}-{parts[1].split(':')[0]}"
            else:
                self.transcription_language = "en-US"
        else:
            self.transcription_language = "en-US"
        
        self.connection: Optional["VoiceLiveConnection"] = None
        self.audio_processor: Optional[AudioProcessor] = None
        self.session_ready = False
        self._active_response = False
        self._response_api_done = False
        self._stop_requested = False
        self._assistant_transcript_buffer = ""
        
        # Tool call tracking
        self._pending_tool_calls: Dict[str, Dict[str, Any]] = {}
        self._tool_call_arguments: Dict[str, str] = {}

    def stop(self):
        """Request graceful stop."""
        self._stop_requested = True
        print("\n🛑 Stop requested, shutting down...")

    async def start(self):
        """Start the voice assistant session."""
        try:
            logger.info("Connecting to VoiceLive API with model %s", self.model)

            async with connect(
                endpoint=self.endpoint,
                credential=self.credential,
                model=self.model,
            ) as connection:
                conn = connection
                self.connection = conn

                ap = AudioProcessor(conn)
                self.audio_processor = ap

                await self._setup_session()
                ap.start_playback()

                logger.info("RAG Voice assistant ready!")
                print("\n" + "=" * 60)
                print("🎤 RAG VOICE ASSISTANT READY")
                print("Ask questions about your knowledge base!")
                print("The assistant will search and cite sources.")
                print("=" * 60 + "\n")

                await self._process_events()
        finally:
            if self.audio_processor:
                self.audio_processor.shutdown()
            if self.rag_tools:
                await self.rag_tools.close()

    async def _setup_session(self):
        """Configure the VoiceLive session with RAG tools."""
        logger.info("Setting up RAG voice session...")

        # Configure voice
        voice_config: Union[AzureStandardVoice, str]
        if "-" in self.voice:
            voice_config = AzureStandardVoice(
                name=self.voice,
                temperature=self.voice_temperature
            )
        else:
            voice_config = self.voice

        # VAD configuration
        vad_type = self.vad_config.get("type", "azure_semantic_vad")
        if vad_type == "azure_semantic_vad":
            turn_detection_config = AzureSemanticVad(
                threshold=self.vad_config.get("threshold", 0.5),
                prefix_padding_ms=self.vad_config.get("prefix_padding_ms", 300),
                speech_duration_ms=self.vad_config.get("speech_duration_ms", 80),
                silence_duration_ms=self.vad_config.get("silence_duration_ms", 500),
            )
        else:
            turn_detection_config = ServerVad(
                threshold=self.vad_config.get("threshold", 0.8),
                prefix_padding_ms=self.vad_config.get("prefix_padding_ms", 200),
                silence_duration_ms=self.vad_config.get("silence_duration_ms", 1000),
            )

        # Input transcription
        input_transcription_config = None
        if self.input_audio_transcription_enabled:
            input_transcription_config = AudioInputTranscriptionOptions(
                model=self.input_audio_transcription_model or "azure-speech",
                language=self.transcription_language,
            )

        # Get tool schemas from RAG manager
        tools = self.rag_tools.get_tool_schemas()
        logger.info(f"Configured {len(tools)} RAG tools")

        session_config = RequestSession(
            modalities=[Modality.TEXT, Modality.AUDIO],
            instructions=self.instructions,
            voice=voice_config,
            input_audio_format=InputAudioFormat.PCM16,
            output_audio_format=OutputAudioFormat.PCM16,
            turn_detection=turn_detection_config,
            input_audio_echo_cancellation=AudioEchoCancellation(),
            input_audio_noise_reduction=AudioNoiseReduction(type="azure_deep_noise_suppression"),
            input_audio_transcription=input_transcription_config,
            tools=tools,
        )

        conn = self.connection
        assert conn is not None
        await conn.session.update(session=session_config)
        logger.info("Session configuration sent with RAG tools")

    async def _process_events(self):
        """Process events from VoiceLive."""
        try:
            conn = self.connection
            assert conn is not None
            async for event in conn:
                if self._stop_requested:
                    break
                await self._handle_event(event)
        except asyncio.CancelledError:
            logger.info("Event processing cancelled")
            raise
        except Exception:
            logger.exception("Error processing events")
            raise

    async def _handle_event(self, event):
        """Handle different types of events from VoiceLive."""
        ap = self.audio_processor
        conn = self.connection
        assert ap is not None and conn is not None

        if event.type == ServerEventType.SESSION_UPDATED:
            logger.info("Session ready: %s", event.session.id)
            self.session_ready = True
            ap.start_capture()

        elif event.type == ServerEventType.INPUT_AUDIO_BUFFER_SPEECH_STARTED:
            logger.info("User started speaking")
            print("🎤 Listening...")
            ap.skip_pending_audio()
            if self._active_response and not self._response_api_done:
                try:
                    await conn.response.cancel()
                except Exception as e:
                    if "no active response" not in str(e).lower():
                        logger.warning("Cancel failed: %s", e)

        elif event.type == ServerEventType.INPUT_AUDIO_BUFFER_SPEECH_STOPPED:
            print("🤔 Processing...")

        elif event.type == ServerEventType.RESPONSE_CREATED:
            self._active_response = True
            self._response_api_done = False
            self._assistant_transcript_buffer = ""

        elif event.type == ServerEventType.RESPONSE_AUDIO_DELTA:
            ap.queue_audio(event.delta)

        elif event.type == ServerEventType.RESPONSE_AUDIO_DONE:
            print("🎤 Ready for next input...")

        elif event.type == ServerEventType.RESPONSE_DONE:
            self._active_response = False
            self._response_api_done = True
            
            # Check if there are pending tool calls to process
            if self._pending_tool_calls:
                await self._process_pending_tool_calls()

        # Handle function/tool calls
        elif event.type == ServerEventType.RESPONSE_OUTPUT_ITEM_ADDED:
            if hasattr(event, 'item') and hasattr(event.item, 'type'):
                if event.item.type == "function_call":
                    item = event.item
                    call_id = getattr(item, 'call_id', None)
                    if call_id:
                        self._pending_tool_calls[call_id] = {
                            "name": getattr(item, 'name', None),
                            "item_id": getattr(item, 'id', None),
                        }
                        self._tool_call_arguments[call_id] = ""
                        logger.info(f"Tool call started: {item.name} ({call_id})")

        elif event.type == ServerEventType.RESPONSE_FUNCTION_CALL_ARGUMENTS_DELTA:
            call_id = getattr(event, 'call_id', None)
            delta = getattr(event, 'delta', '')
            if call_id and call_id in self._tool_call_arguments:
                self._tool_call_arguments[call_id] += delta

        elif event.type == ServerEventType.RESPONSE_FUNCTION_CALL_ARGUMENTS_DONE:
            call_id = getattr(event, 'call_id', None)
            if call_id and call_id in self._pending_tool_calls:
                self._pending_tool_calls[call_id]["arguments"] = self._tool_call_arguments.get(call_id, "{}")
                logger.info(f"Tool call arguments complete: {call_id}")

        # Transcription events
        elif event.type == ServerEventType.RESPONSE_AUDIO_TRANSCRIPT_DELTA:
            delta = getattr(event, 'delta', None)
            if delta:
                self._assistant_transcript_buffer += delta

        elif event.type == ServerEventType.RESPONSE_AUDIO_TRANSCRIPT_DONE:
            transcript = getattr(event, 'transcript', None) or self._assistant_transcript_buffer
            if transcript:
                print(f"🤖 Assistant: {transcript}")
            self._assistant_transcript_buffer = ""

        elif event.type == ServerEventType.CONVERSATION_ITEM_INPUT_AUDIO_TRANSCRIPTION_COMPLETED:
            transcript = getattr(event, 'transcript', None)
            if transcript:
                print(f"📝 You said: {transcript}")

        elif event.type == ServerEventType.ERROR:
            msg = event.error.message
            if "no active response" not in msg.lower():
                print(f"❌ Error: {msg}")
                logger.error("VoiceLive error: %s", msg)

    async def _process_pending_tool_calls(self):
        """Process all pending tool calls."""
        conn = self.connection
        assert conn is not None
        
        tool_handlers = self.rag_tools.get_tool_handlers()
        
        for call_id, call_info in list(self._pending_tool_calls.items()):
            tool_name = call_info.get("name")
            arguments_str = call_info.get("arguments", "{}")
            
            if not tool_name:
                continue
            
            handler = tool_handlers.get(tool_name)
            if not handler:
                logger.warning(f"No handler for tool: {tool_name}")
                continue
            
            try:
                args = json.loads(arguments_str)
            except json.JSONDecodeError:
                args = {}
            
            print(f"🔧 Calling tool: {tool_name}")
            logger.info(f"Executing tool: {tool_name} with args: {args}")
            
            try:
                result = await handler(args)
                result_text = result.to_text()
                
                # Send tool result back to VoiceLive
                output_item = FunctionCallOutputItem(
                    call_id=call_id,
                    output=result_text,
                )
                await conn.conversation.item.create(item=output_item)
                await conn.response.create()
                
                logger.info(f"Tool result sent for {tool_name}")
                
            except Exception as e:
                logger.exception(f"Tool execution failed: {tool_name}")
                # Send error result
                error_output = FunctionCallOutputItem(
                    call_id=call_id,
                    output=json.dumps({"error": str(e)}),
                )
                await conn.conversation.item.create(item=error_output)
                await conn.response.create()
        
        # Clear processed calls
        self._pending_tool_calls.clear()
        self._tool_call_arguments.clear()


print("✅ RAGVoiceAssistant class defined")

## 4. Check Audio System

In [ ]:
def check_audio_system():
    """Check if audio input/output devices are available."""
    try:
        p = pyaudio.PyAudio()
        
        input_devices = [
            i for i in range(p.get_device_count())
            if cast(Union[int, float], p.get_device_info_by_index(i).get("maxInputChannels", 0) or 0) > 0
        ]
        
        output_devices = [
            i for i in range(p.get_device_count())
            if cast(Union[int, float], p.get_device_info_by_index(i).get("maxOutputChannels", 0) or 0) > 0
        ]
        
        p.terminate()

        if not input_devices:
            print("❌ No audio input devices found.")
            return False
        if not output_devices:
            print("❌ No audio output devices found.")
            return False
        
        print(f"✅ Found {len(input_devices)} input device(s) and {len(output_devices)} output device(s)")
        return True
        
    except Exception as e:
        print(f"❌ Audio system check failed: {e}")
        return False

audio_ok = check_audio_system()

## 5. Run Voice Assistant Function

In [ ]:
async def run_rag_voice_assistant():
    """Run the RAG voice assistant."""
    # Validate credentials
    if not API_KEY and not USE_TOKEN_CREDENTIAL:
        print("❌ Error: No authentication provided")
        return

    # Create credential
    credential: Union[AzureKeyCredential, AsyncTokenCredential]
    if USE_TOKEN_CREDENTIAL:
        credential = AzureCliCredential()
        print("🔐 Using Azure CLI credential")
    else:
        credential = AzureKeyCredential(API_KEY)
        print("🔑 Using API key credential")

    # Create RAG tools based on selected backend
    rag_tools: Union[RAGToolManager, RedisRAGToolManager]
    
    if RAG_BACKEND == "redis":
        # Use Redis backend
        rag_tools = create_redis_rag_tools_from_env()
        backend_name = "Azure Managed Redis"
        if rag_tools.redis_client:
            print(f"✅ RAG tools initialized with Redis index: {REDIS_INDEX_NAME}")
        else:
            print("⚠️ Redis RAG not configured - assistant will work without knowledge base")
    else:
        # Use Azure AI Search backend (default)
        rag_tools = create_rag_tools_from_env()
        backend_name = "Azure AI Search"
        if rag_tools.search_client:
            print(f"✅ RAG tools initialized with AI Search index: {SEARCH_INDEX}")
        else:
            print("⚠️ AI Search RAG not configured - assistant will work without knowledge base")

    # Create and start voice assistant
    assistant = RAGVoiceAssistant(
        endpoint=ENDPOINT,
        credential=credential,
        model=MODEL,
        voice=VOICE,
        instructions=INSTRUCTIONS,
        rag_tools=rag_tools,
        voice_temperature=VOICE_TEMPERATURE,
        input_audio_transcription_enabled=INPUT_AUDIO_TRANSCRIPTION_ENABLED,
        input_audio_transcription_model=INPUT_AUDIO_TRANSCRIPTION_MODEL,
        vad_config=VAD_CONFIG,
    )

    print("\n🎙️  RAG Voice Assistant with Azure VoiceLive SDK")
    print("=" * 50)
    print(f"🗄️  RAG Backend: {backend_name}")
    if RAG_BACKEND == "redis":
        print(f"📚 Knowledge base: {REDIS_INDEX_NAME or 'Not configured'}")
    else:
        print(f"📚 Knowledge base: {SEARCH_INDEX or 'Not configured'}")
        print(f"🔍 Vector search: {'Enabled' if USE_VECTOR_SEARCH else 'Disabled'}")

    try:
        await assistant.start()
    except KeyboardInterrupt:
        print("\n👋 Voice assistant shut down. Goodbye!")
    except asyncio.CancelledError:
        print("\n👋 Voice assistant interrupted. Goodbye!")
    except Exception as e:
        print(f"❌ Fatal Error: {e}")
        raise

print("✅ run_rag_voice_assistant() defined")

## 6. Configuration & Run

Update the settings below and run this cell to start the RAG voice assistant.

In [ ]:
# ============================================================
# VOICELIVE CONFIGURATION
# ============================================================

API_KEY = os.environ.get("AZURE_VOICELIVE_API_KEY")
ENDPOINT = os.environ.get("AZURE_VOICELIVE_ENDPOINT",
                          "https://your-resource.services.ai.azure.com/")
MODEL = "gpt-4o"

# Voice Configuration
VOICE = "zh-CN-Xiaochen:DragonHDLatestNeural"
VOICE_TEMPERATURE = 0.8

# Transcription
INPUT_AUDIO_TRANSCRIPTION_ENABLED = True
INPUT_AUDIO_TRANSCRIPTION_MODEL = "azure-speech"

# VAD Configuration
VAD_CONFIG: Dict = {
    "type": "azure_semantic_vad",
    "threshold": 0.3,
    "prefix_padding_ms": 300,
    "silence_duration_ms": 500,
    "speech_duration_ms": 80,
}

# Authentication
USE_TOKEN_CREDENTIAL = False

# ============================================================
# RAG BACKEND SELECTION
# ============================================================
# Choose your RAG backend: "ai_search" or "redis"

RAG_BACKEND = "redis"  # Options: "ai_search", "redis"

# ============================================================
# AZURE AI SEARCH CONFIGURATION (when RAG_BACKEND = "ai_search")
# ============================================================

SEARCH_ENDPOINT = os.environ.get("AZURE_SEARCH_ENDPOINT")
SEARCH_INDEX = os.environ.get("AZURE_SEARCH_INDEX")
SEARCH_API_KEY = os.environ.get("AZURE_SEARCH_API_KEY")

# AI Search specific configuration
SEMANTIC_CONFIGURATION = os.environ.get("AZURE_SEARCH_SEMANTIC_CONFIGURATION", "default")
IDENTIFIER_FIELD = os.environ.get("AZURE_SEARCH_IDENTIFIER_FIELD", "id")
CONTENT_FIELD = os.environ.get("AZURE_SEARCH_CONTENT_FIELD", "content")
EMBEDDING_FIELD = os.environ.get("AZURE_SEARCH_EMBEDDING_FIELD", "embedding")
TITLE_FIELD = os.environ.get("AZURE_SEARCH_TITLE_FIELD")
USE_VECTOR_SEARCH = os.environ.get("AZURE_SEARCH_USE_VECTOR", "true").lower() == "true"

# ============================================================
# AZURE MANAGED REDIS CONFIGURATION (when RAG_BACKEND = "redis")
# ============================================================

REDIS_HOST = os.environ.get("AZURE_REDIS_HOST", "xle-redis2.eastus.redis.azure.net")
REDIS_PORT = int(os.environ.get("AZURE_REDIS_PORT", "10000"))
REDIS_PASSWORD = os.environ.get("AZURE_REDIS_PASSWORD")
REDIS_INDEX_NAME = os.environ.get("AZURE_REDIS_INDEX_NAME", "product_index")

# ============================================================
# SYSTEM INSTRUCTIONS (Optimized for Speech)
# ============================================================

INSTRUCTIONS = """
你是一个友好的语音助手，可以访问知识库来回答用户的问题。

语音对话规范：
- 使用自然、口语化的中文表达，就像在和朋友聊天一样
- 不要使用项目符号、编号列表或特殊格式符号
- 用简短的句子，适当停顿，让听众容易理解
- 使用过渡词让回答更流畅，比如"首先"、"另外"、"总的来说"

数字和单位的处理：
- 保持数字的阿拉伯数字形式，如 6.78 英寸、2448x1080、165Hz
- 语音合成系统会自动将数字转换为自然的语音
- 不要手动将数字转写成中文，如不要写"六点七八英寸"

回答问题时：
- 先使用 internal_search 工具在知识库中搜索相关信息
- 根据搜索结果简洁地回答问题
- 使用 report_grounding 工具记录引用的来源
- 如果找不到相关信息，友好地告诉用户，并询问是否需要其他帮助

保持回答简洁明了，像正常对话一样自然。
"""

# ============================================================
# STATUS DISPLAY
# ============================================================

print("📍 VoiceLive Configuration:")
print(f"   Endpoint: {ENDPOINT}")
print(f"   Model: {MODEL}")
print(f"   Voice: {VOICE}")
print(f"   API Key: {'Set' if API_KEY else 'Not set'}")
print()
print(f"🗄️  RAG Backend: {RAG_BACKEND.upper()}")
print()

if RAG_BACKEND == "redis":
    print("🔴 Azure Managed Redis Configuration:")
    print(f"   Host: {REDIS_HOST}:{REDIS_PORT}")
    print(f"   Password: {'Set' if REDIS_PASSWORD else 'Not set'}")
    print(f"   Index: {REDIS_INDEX_NAME}")
else:
    print("🔍 Azure AI Search Configuration:")
    print(f"   Endpoint: {SEARCH_ENDPOINT or 'Not set'}")
    print(f"   Index: {SEARCH_INDEX or 'Not set'}")
    print(f"   API Key: {'Set' if SEARCH_API_KEY else 'Not set'}")
    print(f"   Vector Search: {'Enabled' if USE_VECTOR_SEARCH else 'Disabled'}")
    print(f"   Semantic Config: {SEMANTIC_CONFIGURATION}")

# ============================================================
# RUN THE VOICE ASSISTANT
# ============================================================

if audio_ok:
    try:
        await run_rag_voice_assistant()
    except asyncio.CancelledError:
        print("\n👋 Session ended. Goodbye!")
else:
    print("⚠️ Please fix audio issues before running the voice assistant.")

## 7. View Cited Sources

After a conversation, you can view the sources that were cited.

In [ ]:
# This cell can be run after the conversation to see cited sources
# Note: You'll need to keep a reference to the rag_tools object

# Example:
# sources = rag_tools.get_cited_sources()
# print(f"Sources cited during conversation: {sources}")

print("Run the voice assistant first, then check cited sources here.")